In [32]:
import os

In [1]:
%pwd

'e:\\Diabetes_classifier\\research'

In [3]:
import os

In [4]:
os.chdir('../')

In [67]:
%pwd

'e:\\Diabetes_classifier'

In [69]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class Model_updationConfig:
    root_dir:Path
    updated_model_dir :Path
    criterion :str
    splitter :str 
    min_samples_split :int
    min_samples_leaf  :int


    

In [71]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn.model_selection import GridSearchCV,KFold
from imblearn.over_sampling import SMOTE

In [72]:
from src.DIABETES_CLASSIFIER.constants import config_file_path
from src.DIABETES_CLASSIFIER.constants import param_file_path
from src.DIABETES_CLASSIFIER.utils.common import create_directories,read_yaml

In [75]:

class Configuratoration:
    def __init__(self,config=config_file_path,params=param_file_path):
        self.config=read_yaml(config)
        self.params=read_yaml(params)

        create_directories([self.config.artifacts_root])

    def get_updated_model_config(self)->Model_updationConfig:
        data=self.config.data_ingestion
        transformation=self.config.data_transformation
        model=self.config.data_trainer
        update_model=self.config.Model_updation
        params=self.params.Decision_TreeClassifier
      

        create_directories([update_model.root_dir])

        

        model_updation=Model_updationConfig(
            root_dir=update_model.root_dir,
            updated_model_dir=update_model.updated_model_dir,
            splitter=params.splitter,
            min_samples_leaf=params.min_samples_leaf,
            min_samples_split=params.min_samples_split,criterion=params.criterion)
        
        return (model_updation,data,transformation,model,data)




In [86]:
from src.DIABETES_CLASSIFIER.utils.common import load_model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


import pandas as pd

In [92]:
class Model_updation:
    def __init__(self,model_updation,data,transformation,model):
        self.config=model_updation
        self.transformations=transformation
        self.model=model
        self.data=data

    def get_updated_model(self):

        model=load_model(self.model.model_dir)
        preprocessor=load_model(Path('artifacts\data_transformation\preprocessor.pkl'))

        df=pd.read_csv('artifacts\\data_ingestion\\train.csv')

        X_train,Y_train=(df.iloc[:,:-1],df.iloc[:,-1])

        params_grid={'classifier__splitter':self.config.splitter,
            'classifier__min_samples_leaf':self.config.min_samples_leaf,
            'classifier__min_samples_split':self.config.min_samples_split,
            'classifier__criterion':self.config.criterion}
        
        param_grids = {'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [None, 5, 10],
    'classifier__min_samples_split': [1, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4]}
        




        clf=imbPipeline(steps=[
           ('preprocessor',preprocessor),
           ('sm',SMOTE(random_state=11)),
           ('classifier',RandomForestClassifier())
           ])
        grid_model=GridSearchCV(clf,param_grid=param_grids,cv=5)

        grid_model.fit(X_train,Y_train)

        return grid_model.best_params_

In [93]:
from src.DIABETES_CLASSIFIER.components.data_transformation import Data_Transformation

In [ ]:
config=Configuratoration()
model_updation,data,transformation,model,data=config.get_updated_model_config()


transformation=Data_Transformation(config=transformation,ingestion=data)

x_train,x_test,y_train,y_test=transformation.data_tranformation_initiated()

model=Model_updation(model_updation=model_updation,transformation=transformation,data=data,model=model)

print(x_train.shape)
print(y_train.shape)

model.get_updated_model()